## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

**Задание 1** Градиентного спуска;

**Задание 2** Стохастического градиентного спуска.

Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором.

In [1]:
import numpy as np
from sklearn.base import BaseEstimator

In [2]:
class LinearReg(BaseEstimator):
    def __init__(self, gd_type='full', tolerance=1e-4, max_iter=1000, w0=None, alpha=1e-3, eta=1e-2, reg=None):
        """
        gd_type: 'full' or 'stochastic' or 'momentum'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        reg: ridge regularization parameter, if it exists
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.reg = reg
        self.loss_history = None  # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        self.loss_history = []  
        tmp_arr = np.random.uniform(-1/(2*X.shape[0]), 1/(2*X.shape[0]), X.shape[1])  # Uniform init with values ~ 0
        self.w = self.w0.copy() if self.w0 is not None else tmp_arr
        
        for it in range(self.max_iter):
            self.loss_history.append(self.calc_loss(X,y))
            new_w = None
            
            if self.gd_type == 'full':
                new_w = self.w - self.eta * self.calc_gradient(X, y)
                
            if self.gd_type == 'stochastic':
                rand_index = np.random.randint(0, y.shape[0])
                new_w = self.w - self.eta * self.calc_gradient(X[rand_index], y[rand_index])
            
            if np.linalg.norm(new_w - self.w, 2) < self.tolerance:
                break
                
            self.w = new_w
        
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        
        if (self.reg is not None): 
            w_norm = sum(map(lambda i: i * i, w))
            return X @ self.w.T + self.reg * w_norm
            
        return X @ self.w.T
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        if (self.reg is not None):   
            return 1/X.shape[0] * X.T.dot(X @ self.w.T - y.T) + self.reg/X.shape[0] * self.w.T
        
        return 1/X.shape[0] * X.T.dot(X @ self.w.T - y.T)
            
    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        tmp = X @ self.w.T - y.T
        if (self.reg is not None): 
            w_norm = sum(map(lambda i: i * i, w)) 
            return 1/(2 * X.shape[0]) * tmp * tmp.T + self.reg/(2 * X.shape[0]) * w_norm
        
        return 1/(2 * X.shape[0]) * tmp * tmp.T

**Задание 3**. 
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('NY taxi_train.csv')
data.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [5]:
X = data.drop('trip_duration', axis=1)
y = np.log1p(data['trip_duration'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

**Задание 4**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

Выделим хотя бы какие-то признаки по времени из предыдущего задания, а также добавим сразу константый, так как наша регрессия по умолчанию его не добавляет:

In [6]:
import datetime

import warnings
warnings.filterwarnings('ignore')

In [7]:
extract_datetime = lambda row: datetime.datetime.strptime(row['pickup_datetime'], "%Y-%m-%d %H:%M:%S")
X_train['pickup_datetime'] = data.apply(extract_datetime, axis=1)
X_test['pickup_datetime'] = data.apply(extract_datetime, axis=1)

X_train['year_day'] = X_train.apply(lambda row: row['pickup_datetime'].timetuple().tm_yday, axis=1)
X_test['year_day'] = X_test.apply(lambda row: row['pickup_datetime'].timetuple().tm_yday, axis=1)

X_train['weekday'] = X_train.apply(lambda row: row['pickup_datetime'].weekday(), axis=1)
X_test['weekday'] = X_test.apply(lambda row: row['pickup_datetime'].weekday(), axis=1)

X_train['hour'] =  X_train.apply(lambda row: row['pickup_datetime'].hour, axis=1)
X_test['hour'] =  X_test.apply(lambda row: row['pickup_datetime'].hour, axis=1)

X_train['month'] =  X_train.apply(lambda row: row['pickup_datetime'].month, axis=1)
X_test['month'] =  X_test.apply(lambda row: row['pickup_datetime'].month, axis=1)

X_train['blizzard'] = ((22 <= X_train.loc[:,'year_day']) & (X_train.loc[:,'year_day'] <= 24)).astype('int64')
X_test['blizzard'] =  ((22 <= X_test.loc[:,'year_day']) & (X_test.loc[:,'year_day'] <= 24)).astype('int64')

X_train['summer_anomaly'] =  ((149 <= X_train.loc[:,'year_day']) & (X_train.loc[:,'year_day'] <= 150)).astype('int64')
X_test['summer_anomaly'] =  ((149 <= X_test.loc[:,'year_day']) & (X_test.loc[:,'year_day'] <= 150)).astype('int64')

X_train['const'] = 1
X_test['const'] = 1

In [8]:
X_train.drop(
    ['id', 
     'vendor_id', 
     'pickup_datetime', 'dropoff_datetime', 
     'passenger_count', 
     'pickup_longitude', 'pickup_latitude',
     'dropoff_longitude', 'dropoff_latitude',
     'store_and_fwd_flag'
    ], axis=1, inplace=True)
X_test.drop(
    ['id', 
     'vendor_id', 
     'pickup_datetime', 'dropoff_datetime', 
     'passenger_count', 
     'pickup_longitude', 'pickup_latitude',
     'dropoff_longitude', 'dropoff_latitude',
     'store_and_fwd_flag'
    ], axis=1, inplace=True)

In [9]:
X_train.head(1)

,year_day,weekday,hour,month,blizzard,summer_anomaly,const
430252,46,0,0,2,0,0,1


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021050 entries, 430252 to 305711
Data columns (total 7 columns):
year_day          1021050 non-null int64
weekday           1021050 non-null int64
hour              1021050 non-null int64
month             1021050 non-null int64
blizzard          1021050 non-null int64
summer_anomaly    1021050 non-null int64
const             1021050 non-null int64
dtypes: int64(7)
memory usage: 62.3 MB


In [11]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

In [12]:
def mse(bar_y, y):
    return 1 / y.shape[0] * (bar_y - y) @ (bar_y - y).T

def r2(bar_y, y):
    return 1 - mse(bar_y, y) / mse(y.mean(), y)

mse_scorer = make_scorer(
    mse,
    greater_is_better=False
)

r2_scorer = make_scorer(
    r2,
    greater_is_better=False
)

In [13]:
categorical_features = ['weekday', 'hour', 'month', 'blizzard', 'summer_anomaly']
numeric_features = ['year_day']

y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
column_transformer = ColumnTransformer([
                                        ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical_features),
                                        ('scaling', StandardScaler(), numeric_features)
                                       ])

gd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='full', max_iter=6666))
                          ])

In [15]:
warnings.filterwarnings('ignore')

In [16]:
%%time
model = gd_pipeline.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Test MSE = %.4f' % mse(y_pred, y_test))
y_pred = model.predict(X_test)
print('Test R2 = %.4f' % r2(y_pred, y_test))
print('-----')

Test MSE = 0.6507
Test R2 = -0.0334
-----
CPU times: user 3min 28s, sys: 40.5 s, total: 4min 8s
Wall time: 4min 29s


Будем увеличить eta с тем же фиксированным числом иттераций и смотреть только на ошибки:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%%time
for eta in [1e-1, 1e-3, 2e-2, 3e-2, 5e-2]:
    gd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='full', max_iter=6666, eta=eta))
                          ])
    
    model = gd_pipeline.fit(X_train, y_train)

    print(f'eta = {eta}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('-----')

> The kernel appears to have died. It will restart automatically.
<br>*Либо мой PC не тянет, либо я не понял, что не так (stackover не помог).*

Будем изменять max_iter при наилучшей eta и смотреть только на ошибки:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%%time
for n_iter in [7500, 5000, 2500, 1000, 100]:
    gd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='full', max_iter=n_iter, eta=))
                          ])
    
    model = gd_pipeline.fit(X_train, y_train)

    print(f'max_iter = {n_iter}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('-----')

Заметим, что ... . Продолжим подбирать  eta:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%%time
for eta in [0.15, 0.2, 0.25, 0.5, 1, 2, 5]:
    rgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='full', max_iter=, eta=eta))
                          ])
    
    model = gd_pipeline.fit(X_train, y_train)

    print(f'eta = {eta}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('-----')

Полученные оптимальные параметры: `eta = , max_iter = `.

Теперь займемся стохастическим градиентным спуском. Для этого сначала убедимся, что он работает:

In [17]:
sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter=2500 , eta=0.1))
                          ])

In [18]:
warnings.filterwarnings('ignore')

In [19]:
%%time
model = sgd_pipeline.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Test MSE = %.4f' % mse(y_pred, y_test))
y_pred = model.predict(X_test)
print('Test R2 = %.4f' % r2(y_pred, y_test))
print('-----')

Test MSE = 0.9270
Test R2 = -0.4723
-----
CPU times: user 7.76 s, sys: 2.91 s, total: 10.7 s
Wall time: 11.9 s


Стоит отметить, что считается все гораздо быстрее. Попробуем поперебирать `eta` на уменьшенном числе итераций:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%%time
for eta in [1e-1, 1e-3, 2e-2, 3e-2, 5e-2]:
    sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter=1000 , eta=eta, tolerance=1e-4))
                          ])
    
    model = sgd_pipeline.fit(X_train, y_train)
    
    print(f'eta = {eta}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('-----')

> The kernel appears to have died. It will restart automatically.
<br>*Либо мой PC не тянет, либо я не понял, что не так (stackover не помог).*

Локальный минимум после двух перезапусков находится около . Будем считать это оптимумом. 
<br>Увеличим число итераций:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%%time
for n_iter in [1500, 2000, 2500, 3000, 3500]:
    sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter=n_iter , eta=))
                          ])
    
    model = sgd_pipeline.fit(X_train, y_train)
    
    print(f'max_iter = {n_iter}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('-----')

Видим, что ... . Теперь будем явно выводить число проведенных итераций:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
for eta in [1e-2, 1e-3, 2e-2, 3e-1, 5e-3]:
    sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter= , eta=eta))
                          ])
    
    print(f'eta = {eta}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('Number of iterations:', len(regressor.loss_history))
    print('-----')

Вывод: ... . Попробуем уменьшить tolerance:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
for eta in [1e-2, 1e-3, 2e-2, 3e-1, 5e-3]:
    sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter= , eta=eta, tolerance=1e-6))
                          ])
    
    print(f'eta = {eta}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('Number of iterations:', len(regressor.loss_history))
    print('-----')

Попробуем увеличить число итераций:

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
for iters in [2500, 3000, 3500, 5000, 7500]:
    sgd_pipeline = Pipeline(steps=[
                            ('ohe_and_scaling', column_transformer),
                            ('regression', LinearReg(gd_type='stochastic', max_iter=n_iter , eta=, tolerance=1e-6)))
                          ])
    
    print(f'max_iter = {n_iter}')
    y_pred = model.predict(X_test)
    print('Test MSE = %.4f' % mse(y_pred, y_test))
    y_pred = model.predict(X_test)
    print('Test R2 = %.4f' % r2(y_pred, y_test))
    print('Number of iterations:', len(regressor.loss_history))
    print('-----')

Пусть оптимальными параметрами будут: `eta = , max_iter = `, `tolerance =`.

Общий вывод: ... .

**Задание 5**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для полного, стохастического градиентного спусков. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

In [ ]:
regressor = LinearReg(gd_type='full', max_iter=2500, eta=0.1) # Возьмем оптимальные параметры
regressor.fit(X_train, y_train)

full_loss_history = regressor.loss_history

> The kernel appears to have died. It will restart automatically.
<br>*Либо мой PC не тянет, либо я не понял, что не так (stackover не помог).*

In [ ]:
regressor = LinearReg(gd_type='stochastic', max_iter=3500, eta=0.02, tolerance=1e-6) # Возьмем оптимальные параметры
regressor.fit(X_train, y_train)

stoch_loss_history = regressor.loss_history

In [ ]:
print(len(full_loss_history), len(stoch_loss_history))

In [ ]:
pd.Series(full_loss_history).describe()

In [ ]:
pd.Series(stoch_loss_history).describe()

In [ ]:
def CompareLoss(loss1, loss2):
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))
    
    ax.set_title('Comparing full GD and stochastic GD')
    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    
    ax.set_ylim((0, 20))
    
    ax.plot(loss1, color='blue', label='full GD')
    ax.plot(loss2, color='red', label='stochastic GD')
    ax.legend()
    
    plt.show()

In [ ]:
CompareLoss(full_loss_history, stoch_loss_history)

Общий вывод: ... .